In [0]:

pip install Faker

In [0]:
dbutils.library.restartPython()

# Products datasets Preparation

In [0]:
catalog = "workspace"
schema = dbName = db = "customers"
volume_name = "raw_data"

spark.sql(f'CREATE CATALOG IF NOT EXISTS `{catalog}`')
spark.sql(f'USE CATALOG `{catalog}`')
spark.sql(f'CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{schema}`')
spark.sql(f'USE SCHEMA `{schema}`')
spark.sql(f'CREATE VOLUME IF NOT EXISTS `{catalog}`.`{schema}`.`{volume_name}`')
volume_folder =  f"/Volumes/{catalog}/{db}/{volume_name}"


try:
  dbutils.fs.ls(volume_folder + "/products")
except:
  print(f"folder doesn't exist, generating the data under {volume_folder}...")
  from pyspark.sql import functions as F
  from faker import Faker
  import uuid
  import random

  fake = Faker()
  fake_product_id = F.udf(lambda: str(uuid.uuid4()))
  fake_product_name = F.udf(fake.word)
  fake_category = F.udf(lambda: random.choice(["Electronics", "Clothing", "Books", "Home", "Toys", "Sports"]))
  fake_price = F.udf(lambda: round(random.uniform(5, 2000), 2))
  fake_stock = F.udf(lambda: random.randint(0, 500))
  fake_description = F.udf(fake.sentence)

  products_df = spark.range(0, 10000).repartition(10)
  products_df = products_df.withColumn("product_id", fake_product_id())
  products_df = products_df.withColumn("product_name", fake_product_name())
  products_df = products_df.withColumn("category", fake_category())
  products_df = products_df.withColumn("price", fake_price())
  products_df = products_df.withColumn("stock", fake_stock())
  products_df = products_df.withColumn("description", fake_description())

  products_df.select("product_id", "product_name", "category", "price", "stock", "description") \
    .repartition(1) \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save(volume_folder + "/products")

